<a href="https://colab.research.google.com/github/quanvu0996/compet/blob/master/kalapa_4student_net2_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, DenseFeatures, Dropout, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.metrics import AUC, Precision, Recall
import tensorflow.keras.backend as K
from tensorflow import feature_column as fc
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

%matplotlib inline

In [22]:
print(pd.__version__)
print(tf.__version__)

1.0.5
2.3.0


## Load data

In [23]:
submision_path = '/content/drive/My Drive/Data/colabs_data/kalapa_4students/simple_submission.csv'
train_path = '/content/drive/My Drive/Data/colabs_data/kalapa_4students/train.csv'
test_path = '/content/drive/My Drive/Data/colabs_data/kalapa_4students/test.csv'

In [24]:
def load_data(train_path, test_path, label_col = 'label'):
    train_set = pd.read_csv(train_path)
    test_set = pd.read_csv(test_path)
    return train_set, test_set

train_set, test_set = load_data( train_path, test_path )
cols = train_set.columns

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [25]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53030 entries, 0 to 53029
Columns: 195 entries, id to Field_82
dtypes: float64(133), int64(3), object(59)
memory usage: 78.9+ MB


### Data trasforming

In [26]:
# train_set[cols_categorical].head()
# train_set.Field_61.unique()

In [27]:
# len(train_set.Field_45.unique())
# cols_id

In [28]:
# len(train_set.homeTownCity.unique())

In [29]:
# train_set.C_endDate.unique()
# train_set.homeTownCity.hist()
# print(cols_docs)

In [30]:
train_set[cols_date].head()

,Field_1,Field_2,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_14,Field_15,Field_25,Field_32,Field_33,Field_34,Field_35,Field_40,Field_43,Field_44,F_startDate,F_endDate,E_startDate,E_endDate,C_startDate,C_endDate,G_startDate,G_endDate,A_startDate,A_endDate
0,2019-07-24T03:39:02.854Z,2019-07-31T20:10:02Z,2018-12-27,2018-12-27,2019-01-01,2019-07-31,2018-12-27,1.0,2019-07-31,1.0,NaN,2018-12-27,2018-12-28,2022-09-01,20180107,2019-07-31,2019-01-01,2017-01-03T16:06:41.157Z,2019-11-12T04:09:58.381Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-06-11,2019-08-17
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-17,2019-07-17
2,2019-01-17T07:15:26.367Z,2019-01-17T07:17:45Z,2019-01-17,2019-01-17,2019-01-01,2019-12-31,2019-01-17,1.0,NaN,1.0,2019-01-17,2019-01-17,2019-01-17,2023-03-01,20190102,NaN,2019-01-01,2019-01-07T08:29:01.623Z,2019-08-26T09:27:23.191Z,2019-08-18,2019-08-18,2019-08-18,2019-09-11,NaN,NaN,NaN,NaN,2019-09-19,2019-11-05
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Nhãn
cols_label = ['label']
# Đặc trưng binary
cols_fts_binary = [i for i in train_set.select_dtypes(include=['float64','int64']).columns 
                   if len(set(train_set[i].fillna(0)) - {0,1} ) == 0 and i not in cols_label]
# định dang datte yyyy-mm-dd
cols_date = ['Field_1','Field_2','Field_5','Field_6','Field_7','Field_8','Field_9','Field_10','Field_11'
             ,'Field_15','Field_25','Field_32','Field_33','Field_35','Field_40','Field_43','Field_44'
             ,'F_startDate','F_endDate','E_startDate','E_endDate','C_startDate','C_endDate','G_startDate','G_endDate'
             ,'A_startDate','A_endDate']
# định dạng date yyymmdd
cols_date2 = ['ngaySinh', 'Field_34']
# Đặc trưng dạng văn bản
cols_docs = ['Field_46','diaChi','Field_48','Field_49','currentLocationName','homeTownName','Field_56']
# Định danh bản ghi
cols_id = ['id','Field_45'] +[i for i in train_set.select_dtypes(include = ['object']).columns 
                              if len(train_set[i].unique()) >=350 and i not in cols_date + cols_date2+cols_docs]
# Đặc trưng dạng categorical
cols_categorical = [i for i in train_set.select_dtypes(include = ['object']).columns
                    if i not in cols_id + cols_label+cols_date2+cols_date+cols_docs]
# Đặc trưng số
cols_fts_num = [ i for i in test_set.select_dtypes(include=['float64','int64']).columns 
                if i not in cols_id + cols_label + cols_fts_binary + cols_date2+cols_date+cols_docs]

In [82]:
# cols_categorical
cols_date2 = ['ngaySinh', 'Field_34']

In [33]:
# Đặc trưng số có dạng như categorical
cols_num_like_cat = [i for i in cols_fts_num if len(train_set[i].unique()) <= 15 ]
# Đặc trưng số đã kiểm chứng
cols_num = [i for i in cols_fts_num if i not in cols_num_like_cat]

cols_categorical = cols_categorical + cols_num_like_cat
cols_fts_num = [i for i in cols_fts_num if i not in cols_num_like_cat ]

In [34]:
# for i in cols_fts_num :
#   print(i, len(train_set[i].unique()))

Missing values

In [ ]:
train_data = train_set
predict_data = test_set

# Categorical: Fill giá trị null như một giá trị khác
train_data[cols_categorical] = train_set[cols_categorical].astype('object').fillna('#NULL#').applymap(str)
predict_data[cols_categorical] = test_set[cols_categorical].astype('object').fillna('#NULL#').applymap(str)

In [81]:
train_data[col]

0        20180107
1             NaN
2        20190102
3             NaN
4             NaN
           ...   
53025    20191205
53026         NaN
53027    20170709
53028         NaN
53029         NaN
Name: Field_34, Length: 53030, dtype: object

In [100]:
# Date: với dữ liệu ngày thì tạo ra các cột ngày tương ứng và cross colum => embedding để giảm chiều

def trans_date_fts(df, col, format = 'yyyy-mm-dd'):
  '''format là 'yyyy-mm-dd' hoặc 'yyyymmdd' '''

  year_name = 'year_of'+col 
  month_name = 'month_of' + col
  day_name = 'day_of' + col
  if format == 'yyyy-mm-dd':
    df[year_name] = df[col].apply(lambda x: str(x[:4]) if len(str(x)) >8 else '#NULL#')
    df[month_name] = df[col].apply(lambda x: str(x[5:7]) if len(str(x)) >8 else '#NULL#')
    df[day_name] = df[col].apply(lambda x: str(x[8:10]) if len(str(x)) >8 else '#NULL#')
  else:
    df[year_name] = df[col].apply(lambda x: str(x)[:4] if len(str(x)) >8 else '#NULL#')
    df[month_name] = df[col].apply(lambda x: str(x)[4:6] if len(str(x)) >8 else '#NULL#')
    df[day_name] = df[col].apply(lambda x: str(x)[6:8] if len(str(x)) >8 else '#NULL#')

  ft1 = fc.categorical_column_with_vocabulary_list(year_name, df[year_name].unique() )
  ft2 = fc.categorical_column_with_vocabulary_list(month_name, df[month_name].unique() )
  ft3 = fc.categorical_column_with_vocabulary_list(day_name, df[day_name].unique() )
  ft = fc.crossed_column([ft1, ft2, ft3], hash_bucket_size = 366)
  ft = fc.embedding_column(ft, dimension= 64)

  return df, ft, [year_name, month_name, day_name]

fts_col_date = []
date_derivated_fts = []
for col in cols_date:
  train_data, date_fts_list_add, date_fts_name = trans_date_fts(train_data, col,format = 'yyyy-mm-dd')
  fts_col_date.append( date_fts_list_add )
  date_derivated_fts.append(date_fts_name)
for col in cols_date2:
  train_data, date_fts_list_add, date_fts_name = trans_date_fts(train_data, col,format = 'yyyymmdd')
  fts_col_date.append( date_fts_list_add )
  date_derivated_fts.append(date_fts_name)

for col in cols_date:
  predict_data, date_fts_list_add, date_fts_name = trans_date_fts(predict_data, col,format = 'yyyy-mm-dd')
for col in cols_date2:
  predict_data, date_fts_list_add, date_fts_name = trans_date_fts(predict_data, col,format = 'yyyymmdd')


In [85]:
X_train, X_test, y_train, y_test = train_test_split(train_data
                                                   ,train_data['label'].values
                                                   , stratify = train_data['label'].values 
                                                   ,test_size = 0.2
                                                   )

In [87]:
# Numerical: các trường chưa xử lý+ numerical fill giá trị -1

# train_data[cols_fts_num] = train_set[cols_fts_num].fillna(-1)
# predict_data[cols_fts_num] = test_set[cols_fts_num].fillna(-1)
# X_train[cols_fts_num] = (X_train[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()
# X_test[cols_fts_num] = (X_test[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()
# predict_data[cols_fts_num] = (predict_data[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
predict_data = predict_data.fillna(0)

Định nghĩa features

In [92]:
feature_columns = []
fts_cols = cols_fts_binary+cols_fts_num+cols_categorical +cols_date+cols_date2

# Với biến binary, load trực tiếp vào model
for col in cols_fts_binary:
  feature_columns.append(
      fc.numeric_column(col, dtype=tf.float32)
  )
# Với các biến numeric, sử dụng standard scaler + định nghĩa
# X_train[cols_fts_num] = (X_train[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()
# X_test[cols_fts_num] = (X_test[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()

for col in cols_fts_num:
  mean = X_train[col].mean()
  std = X_train[col].std()
  feature_columns.append(
      fc.numeric_column(col,
                        dtype=tf.float32
                        #, normalizer_fn = lambda x: ( x - mean )/ std 
                        ))
# Với các biến categorical, embed với số chiều bằng 1/10 +1 số chiều gốc
def embed_trans(feature_name, vocab):
  ft = fc.categorical_column_with_vocabulary_list(feature_name, vocab)
  ft = fc.embedding_column( ft,dimension= int(len(vocab)/10) +5 )
  return ft
for col in cols_categorical:
  vocabulary = train_set[col].unique()
  feature_columns.append(
      embed_trans(col, vocabulary)
  )


# Xây dựng layer
all_fts_cols = feature_columns +fts_col_date
feature_layer = DenseFeatures(all_fts_cols)

In [93]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(X, y, shuffle=True, batch_size=32):
  ds = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(X))
  ds = ds.batch(batch_size)
  return ds

In [94]:
X_test.head()

,id,label,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16,Field_17,Field_18,Field_19,Field_20,Field_21,Field_22,Field_23,Field_24,Field_25,Field_26,Field_27,Field_28,Field_29,Field_30,Field_31,Field_32,Field_33,Field_34,Field_35,ngaySinh,namSinh,gioiTinh,...,day_ofField_43,year_ofField_44,month_ofField_44,day_ofField_44,year_ofF_startDate,month_ofF_startDate,day_ofF_startDate,year_ofF_endDate,month_ofF_endDate,day_ofF_endDate,year_ofE_startDate,month_ofE_startDate,day_ofE_startDate,year_ofE_endDate,month_ofE_endDate,day_ofE_endDate,year_ofC_startDate,month_ofC_startDate,day_ofC_startDate,year_ofC_endDate,month_ofC_endDate,day_ofC_endDate,year_ofG_startDate,month_ofG_startDate,day_ofG_startDate,year_ofG_endDate,month_ofG_endDate,day_ofG_endDate,year_ofA_startDate,month_ofA_startDate,day_ofA_startDate,year_ofA_endDate,month_ofA_endDate,day_ofA_endDate,year_ofngaySinh,month_ofngaySinh,day_ofngaySinh,year_ofField_34,month_ofField_34,day_ofField_34
44181,44181,0,2019-11-01T11:16:00.437Z,2019-11-01T11:17:20Z,2.0,T1,2019-11-01,2019-11-01,2019-10-01,2019-12-31,2019-11-01,1.0,0,#NULL#,1.0,1.0,2019-11-01,1.0,1.0,0,0.0,3969700.0,4.5,1.0,8.0,1.0,2019-11-01,1.0,0.0,0.0,0.0,1.0,1.0,2019-11-01,2024-02-22,20191004,0,19930306.0,0.0,FEMALE,...,23,2019,06,27,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,2019,09,06,2019,09,21,1993,03,06,#NULL#,#NULL#,#NULL#
29618,29618,0,0,0,#NULL#,#NULL#,0,0,0,0,0,0.0,0,#NULL#,0.0,0.0,0,0.0,0.0,0,0.0,0.0,#NULL#,#NULL#,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,#NULL#,#NULL#,...,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#
48677,48677,1,2018-04-02T12:47:33.015Z,2018-04-02T12:47:33.015Z,2.0,T1,2018-01-03,2018-01-03,2018-01-01,2018-03-31,2018-01-03,1.0,2018-03-31,G8,1.0,1.0,2018-01-03,1.0,1.0,0,0.0,4000000.0,4.5,1.0,15.0,1.0,2018-01-03,1.0,0.0,0.0,3.0,1.0,1.0,2018-01-03,2021-10-01,20170903,2018-03-31,19890317.0,0.0,MALE,...,01,2019,10,02,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,1989,03,17,#NULL#,#NULL#,#NULL#
12224,12224,0,0,0,#NULL#,#NULL#,0,0,0,0,0,0.0,0,#NULL#,0.0,0.0,0,0.0,0.0,0,0.0,0.0,#NULL#,#NULL#,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,#NULL#,#NULL#,...,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,2019,08,28,2019,08,30,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#
33041,33041,1,2019-08-26T02:25:33.646Z,2019-08-31T20:12:27Z,1.0,GH,2018-12-24,2018-12-24,2019-01-01,2019-08-31,2018-12-24,1.0,2019-08-31,G8,1.0,1.0,0,1.0,1.0,Đ0451,0.0,4258600.0,4.5,1.0,5.0,1.0,2018-12-24,1.0,0.0,0.0,2.0,1.0,1.0,2018-12-24,2023-08-01,2.01809e+07,2019-08-31,19890109.0,0.0,MALE,...,24,2019,06,26,2019,09,02,2019,09,02,2019,09,02,2019,09,04,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,#NULL#,2019,09,02,2019,09,02,2019,03,18,2019,09,13,1989,01,09,2018,09,04


In [41]:
# scaler_1 = StandardScaler()
# X_train_scaled = scaler_1.fit_transform(X_train)
# X_test_scaled = scaler_1.transform(X_test)
# X_predict_scaled = scaler_1.transform( test_set[cols_fts_num].fillna(0) )

In [96]:
nets_2 = Sequential(
    [
    feature_layer,
    BatchNormalization (),
    Dense(512, activation ='relu'),
    Dropout(0.25), 
    BatchNormalization (),
    Dense(256, activation ='relu'),
    Dropout(0.25), 
    Dense(128, activation ='relu'),
    Dropout(0.25), 
    BatchNormalization (),
    Dense(64, activation ='relu'),
    Dropout(0.1), 
    Dense(16, activation ='relu'),
    Dense(1, activation ='sigmoid')])
nets_2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc', AUC() ] ) #, Precision(), Recall()] )

In [98]:
batch_size = 50 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(X_train, y_train, batch_size=batch_size)
test_ds = df_to_dataset(X_test, y_test, shuffle=False, batch_size=batch_size)
predict_ds = df_to_dataset(predict_data, predict_data['id'], shuffle=False, batch_size=batch_size)

history = nets_2.fit( train_ds , epochs = 10
                    , validation_data = test_ds)
nets_2.summary()

ValueError: ignored

Dự đoán và xuất file kế quả

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
y_predict_pred= nets_2.predict(predict_ds )
y_predict_pred

In [ ]:
res_df = pd.DataFrame({'id': test_set.id, 'label': y_predict_pred.reshape(y_predict_pred.shape[0])})
res_df.to_csv('submission.csv', index=False)